To execute this notebook, you need to either
 - Download a pre-trained model
 - Train an example model by excecuting the model_training_tutorial.ipynb notebook

In this tutorial we will learn to:
- Load a previously trained model
- Extract DeepPrint features from fingerprint images
- Evaluate the performance of the extracted fixed-length representations

## Embedding extraction

After training the model, we can extract the DeepPrint features for the fingerprint images. This is done by calling the `extract` method of the `DeepPrintExtractor` class.

In [2]:
import os

from flx.extractor.fixed_length_extractor import get_DeepPrint_Tex, get_DeepPrint_TexMinu, DeepPrintExtractor

# Dimension and number of training subjects must be known to load the pre-trained model

# To load the pre-trained model parameters use num_training_subjects=8000
extractor: DeepPrintExtractor = get_DeepPrint_TexMinu(num_training_subjects=140, num_dims=128)

# To load the pre-trained model parameters use
MODEL_DIR: str = os.path.abspath("/home/mt0/22CS60R42/dataset/fvc2006db3_A_model") # Path to the directory containing the model parameters
extractor.load_best_model(MODEL_DIR)

Loaded best model from /home/mt0/22CS60R42/dataset/fvc2006db3_A_model/best_model.pyt


KeyError: "filename 'storages' not found"

Now we need to specify the dataset, for which we want to extract the embeddings

In [2]:
import os

from flx.data.dataset import *
from flx.data.image_loader import SFingeLoader
from flx.data.transformed_image_loader import TransformedImageLoader
from flx.image_processing.binarization import LazilyAllocatedBinarizer
from flx.data.image_helpers import pad_and_resize_to_deepprint_input_size

# NOTE: If this does not work, enter the absolute path to the notebooks/example-dataset directory here! 
DATASET_PATH: str = os.path.abspath("/home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/FVC2006DB2_A")

# We will use the SFingeLoader to load the images from the dataset
image_loader = TransformedImageLoader(
        images=SFingeLoader(DATASET_PATH),
        poses=None,
        transforms=[
            LazilyAllocatedBinarizer(5.0),
            pad_and_resize_to_deepprint_input_size,
        ],
    )

image_dataset: Dataset = Dataset(image_loader, image_loader.ids)

# The second value is for the minutiae branch, which we do not have in this example
texture_embeddings, minutia_embeddings = extractor.extract(image_dataset)

Created IdentifierSet with 140 subjects and a total of 1680 samples.


100%|██████████| 53/53 [12:02<00:00, 13.63s/it]


In [6]:
texture_embeddings = extractor.get_DeepPrint_Tex

[[ 0.04198745 -0.09042273  0.03778943 ...  0.05165511  0.05490642
   0.03112722]
 [ 0.10568812 -0.0844481   0.07519364 ...  0.08187614  0.06651236
  -0.00984057]
 [ 0.07251018 -0.07849421  0.07919195 ...  0.06697464  0.05448965
   0.00292787]
 ...
 [ 0.04042731 -0.0387625   0.02640067 ... -0.03194454  0.06001967
   0.00752258]
 [ 0.00854072 -0.02655505 -0.00838139 ... -0.0006599   0.06759997
  -0.0119191 ]
 [ 0.04564777 -0.02835366  0.00805646 ... -0.02258757  0.06643154
   0.01312804]]


## Benchmarking

To evaluate the embeddings, we want to run a benchmark on them. For this, we must first specify the type of benchmark, and which comparisons should be run.

In [3]:
from flx.scripts.generate_benchmarks import create_verification_benchmark

NUM_IMPRESSIONS_PER_SUBJECT = 12
benchmark = create_verification_benchmark(
    #subjects=list(range(100, image_dataset.num_subjects + 100)),
    subjects=list(range(image_dataset.num_subjects)),
    impressions_per_subject=list(range(NUM_IMPRESSIONS_PER_SUBJECT))
)

100%|██████████| 140/140 [00:00<00:00, 7307.51it/s]


Now we can run the benchmark. To do this, we must first specify the matcher (in our case cosine similarity of the embeddings)

In [8]:
from flx.benchmarks.matchers import CosineSimilarityMatcher
from flx.data.embedding_loader import EmbeddingLoader

# We concatenate texture and minutia embedding vectors
embeddings = EmbeddingLoader.combine(texture_embeddings, minutia_embeddings)
matcher = CosineSimilarityMatcher(EmbeddingLoader.combine(texture_embeddings, minutia_embeddings))

results = benchmark.run(matcher)

print(f"Equal-Error-Rate: {results.get_equal_error_rate()}")

Created IdentifierSet with 140 subjects and a total of 1680 samples.
Created IdentifierSet with 140 subjects and a total of 1680 samples.
Created IdentifierSet with 140 subjects and a total of 1680 samples.
Created IdentifierSet with 140 subjects and a total of 1680 samples.
Created IdentifierSet with 140 subjects and a total of 1680 samples.
Created IdentifierSet with 140 subjects and a total of 1680 samples.


  0%|          | 0/29400 [00:00<?, ?it/s]

100%|██████████| 29400/29400 [00:00<00:00, 55984.20it/s]

Equal-Error-Rate: 0.13839285714285715


To visualize the results, we can plot a DET curve. (Do not wonder if it is empty, probably the model is not trained enough. Take a look at the EER instead.)

In [ ]:
from flx.visualization.plot_DET_curve import plot_verification_results

figure_path = "DET_curve"

# Lists are used to allow for multiple models to be plotted in the same figure
plot_verification_results(figure_path, results=[results], model_labels=["DeepPrint_TexMinu"], plot_title="example-dataset - verification")